In [4]:
import pandas as pd
import numpy as np
from joblib import dump
from sklearn.neighbors import NearestNeighbors
from joblib import dump

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="9e5f390cdd7e44658f17b1b3228c02b2",
                                                           client_secret="a2d7e42715d54e12a3a4e86ba5d93dc0"))

In [ ]:
"""
Primary:
- id (Id of track generated by Spotify)
Numerical:
- acousticness (Ranges from 0 to 1)
- danceability (Ranges from 0 to 1)
- energy (Ranges from 0 to 1)
- duration_ms (Integer typically ranging from 200k to 300k)
- instrumentalness (Ranges from 0 to 1)
- valence (Ranges from 0 to 1)
- popularity (Ranges from 0 to 100)
- tempo (Float typically ranging from 50 to 150)
- liveness (Ranges from 0 to 1)
- loudness (Float typically ranging from -60 to 0)
- speechiness (Ranges from 0 to 1)
- year (Ranges from 1921 to 2020)
Dummy:
- mode (0 = Minor, 1 = Major)
- explicit (0 = No explicit content, 1 = Explicit content)
Categorical:
- key (All keys on octave encoded as values ranging from 0 to 11, starting on C as 0, C# as 1 and so on…)
- artists (List of artists mentioned)
- release_date (Date of release mostly in yyyy-mm-dd format, however precision of date may vary)
- name (Name of the song)"""

In [26]:
urn = 'spotify:track:0cS0A1fUEUd1EW3FcF8AEI'

results = sp.track(urn)
random = pd.json_normalize(results)
random

,artists,available_markets,disc_number,duration_ms,explicit,href,id,is_local,name,popularity,...,album.id,album.images,album.name,album.release_date,album.release_date_precision,album.total_tracks,album.type,album.uri,external_ids.isrc,external_urls.spotify
0,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1,168333,False,https://api.spotify.com/v1/tracks/0cS0A1fUEUd1...,0cS0A1fUEUd1EW3FcF8AEI,False,Keep A Song In Your Soul,11,...,6os2Mv58OYnQClPf7B9E1s,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Crazy Blues: The Best Of Mamie Smith,1920,year,25,album,spotify:album:6os2Mv58OYnQClPf7B9E1s,USSM10309151,https://open.spotify.com/track/0cS0A1fUEUd1EW3...


In [29]:
def song_model(input):
    df = pd.read_csv('suggestor/edited_data.csv')
    # adding direct url to data set by adding url prefix and id
    url = 'http://open.spotify.com/track/' + df['id']
    df['url'] = url
    
    # reordering columns, leaving out ID and release date
    df = df[['artists',  'name', 'url', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'explicit', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'popularity', 'speechiness', 'tempo', 'valence']]
    
   # target set will be both artist and name
    y_set = ['artists', 'name', 'url']

    # droping target from data matrix
    df_data = df.drop(y_set, axis=1)

    # set target
    df_target = df[y_set]

    # fit on data, 12 neighbors
    nn = NearestNeighbors(algorithm='brute', leaf_size =15, n_neighbors=15, n_jobs=-1)
    nn.fit(df_data)

    # sample a song(index) from df_data to use as our query point 
    input_index=input

    # vectorize 
    data_vect = [df_data.iloc[input_index].values]
    neigh_dist, neigh_indices = nn.kneighbors(data_vect)
    indexs = neigh_indices.flat[2:15].tolist()

    #adding url to each track
    output = df_target.iloc[indexs]
    return (output)

In [30]:
input = 'Shake It Off'
index = df.loc[df.isin([input]).any(axis=1)].index.tolist()
index = index[0]
model = song_model(index) 
model

,artists,name,url
92700,TAYLOR SWIFT,Shake It Off,http://open.spotify.com/track/3fthfkkvy9av3q3u...
74252,JESSE MCCARTNEY,Body Language,http://open.spotify.com/track/7lT9KJpfzMTNi970...
92762,J. COLE,Hello,http://open.spotify.com/track/30Chv2SmIry70Ywt...
172017,"DANIEL MERRIWEATHER', 'ADELE",Water And A Flame (feat. Adele),http://open.spotify.com/track/6FZjUIfR6vEEg1bT...
38094,JASON ALDEAN,Burnin' It Down,http://open.spotify.com/track/5HJqpDspKDKwQpLj...
12912,TALKING HEADS,And She Was,http://open.spotify.com/track/5JwK7v6XhG0YgqDu...
11592,CARPENTERS,Christmas Song (Chestnuts Roasting On An Open ...,http://open.spotify.com/track/177bLiZAvMp5FSLQ...
69471,CELTIC FROST,Into the Crypts of Rays,http://open.spotify.com/track/4xQO8MVtFZOW2KJM...
136837,PATRICK BRUEL,J'te l'dis quand même,http://open.spotify.com/track/1Ek1OEhcEw9gRr5f...
72350,BECK,Sexx Laws,http://open.spotify.com/track/1ELY3KTgI83N1k0x...


In [5]:
# reading csv
df = pd.read_csv('suggestor/edited_data.csv')

In [6]:
df

,Unnamed: 0,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0,0.991000,MAMIE SMITH,0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,1,0.643000,SCREAMIN' JAY HAWKINS,0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,2,0.993000,MAMIE SMITH,0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,3,0.000173,OSCAR VELAZQUEZ,0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,4,0.295000,MIXE,0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174384,174384,0.009170,"DJ COMBO', 'SANDER-7', 'TONY T",0.792,147615,0.866,0,46LhBf6TvYjZU2SMvGZAbn,0.000060,6,0.1780,-5.089,0,The One,0,2020-12-25,0.0356,125.972,0.1860,2020
174385,174385,0.795000,ALESSIA CARA,0.429,144720,0.211,0,7tue2Wemjd0FZzRtDrQFZd,0.000000,4,0.1960,-11.665,1,A Little More,0,2021-01-22,0.0360,94.710,0.2280,2021
174386,174386,0.806000,ROGER FLY,0.671,218147,0.589,0,48Qj61hOdYmUCFJbpQ29Ob,0.920000,4,0.1130,-12.393,0,Together,0,2020-12-09,0.0282,108.058,0.7140,2020
174387,174387,0.920000,TAYLOR SWIFT,0.462,244000,0.240,1,1gcyHQpBQ1lfXGdhZmWrHP,0.000000,0,0.1130,-12.077,1,champagne problems,69,2021-01-07,0.0377,171.319,0.3200,2021


In [7]:
# adding direct url to data set by adding url prefix and id
url = 'http://open.spotify.com/track/' + df['id']
df['url'] = url

In [8]:
# reordering columns, leaving out ID and release date
df = df[['artists',  'name', 'url', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'explicit', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'popularity', 'speechiness', 'tempo', 'valence']]

In [6]:
df

,artists,name,url,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
0,MAMIE SMITH,Keep A Song In Your Soul,http://open.spotify.com/track/0cS0A1fUEUd1EW3F...,1920,0.991000,0.598,168333,0.224,0,0.000522,5,0.3790,-12.628,0,12,0.0936,149.976,0.6340
1,SCREAMIN' JAY HAWKINS,I Put A Spell On You,http://open.spotify.com/track/0hbkKFIJm7Z05H8Z...,1920,0.643000,0.852,150200,0.517,0,0.026400,5,0.0809,-7.261,0,7,0.0534,86.889,0.9500
2,MAMIE SMITH,Golfing Papa,http://open.spotify.com/track/11m7laMUgmOKqI3o...,1920,0.993000,0.647,163827,0.186,0,0.000018,0,0.5190,-12.098,1,4,0.1740,97.600,0.6890
3,OSCAR VELAZQUEZ,True House Music - Xavier Santos & Carlos Gomi...,http://open.spotify.com/track/19Lc5SfJJ5O1oaxY...,1920,0.000173,0.730,422087,0.798,0,0.801000,2,0.1280,-7.311,1,17,0.0425,127.997,0.0422
4,MIXE,Xuniverxe,http://open.spotify.com/track/2hJjbsLCytGsnAHf...,1920,0.295000,0.704,165224,0.707,1,0.000246,10,0.4020,-6.036,0,2,0.0768,122.076,0.2990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174384,"DJ COMBO', 'SANDER-7', 'TONY T",The One,http://open.spotify.com/track/46LhBf6TvYjZU2SM...,2020,0.009170,0.792,147615,0.866,0,0.000060,6,0.1780,-5.089,0,0,0.0356,125.972,0.1860
174385,ALESSIA CARA,A Little More,http://open.spotify.com/track/7tue2Wemjd0FZzRt...,2021,0.795000,0.429,144720,0.211,0,0.000000,4,0.1960,-11.665,1,0,0.0360,94.710,0.2280
174386,ROGER FLY,Together,http://open.spotify.com/track/48Qj61hOdYmUCFJb...,2020,0.806000,0.671,218147,0.589,0,0.920000,4,0.1130,-12.393,0,0,0.0282,108.058,0.7140
174387,TAYLOR SWIFT,champagne problems,http://open.spotify.com/track/1gcyHQpBQ1lfXGdh...,2021,0.920000,0.462,244000,0.240,1,0.000000,0,0.1130,-12.077,1,69,0.0377,171.319,0.3200


In [9]:
combined = df['name'] + ' - ' +  df['artists']
track_artist = combined.tolist()

In [11]:
foo = ['Keep A Song In Your Soul', 'I Put A Spell On You', 'Golfing Papa', 'Keep Your Eyes Open']

In [15]:
import json

In [16]:
data = json.dump(track_artist)

TypeError: dump() missing 1 required positional argument: 'fp'

In [10]:
track_artist[:12]

['Keep A Song In Your Soul - MAMIE SMITH',
 "I Put A Spell On You - SCREAMIN' JAY HAWKINS",
 'Golfing Papa - MAMIE SMITH',
 'True House Music - Xavier Santos & Carlos Gomix Remix - OSCAR VELAZQUEZ',
 'Xuniverxe - MIXE',
 'Crazy Blues - 78rpm Version - MAMIE SMITH & HER JAZZ HOUNDS',
 "Don't You Advertise Your Man - MAMIE SMITH",
 'Arkansas Blues - MAMIE SMITH & HER JAZZ HOUNDS',
 'La Chacarera - Remasterizado - FRANCISCO CANARO',
 'Broken Puppet - Original Mix - MEETYA',
 'Oouin - DORVILLE',
 'Desengaño - Remasterizado - FRANCISCO CANARO']

In [32]:
import json

my_details = track_artist

with open('data.json', 'w') as json_file:
    json.dump(my_details, json_file)

In [31]:
len(track_artist)

174389

In [33]:
user = {'firstname': "Mr.", 'lastname': "My Father's Son"}

In [34]:
type(user)

dict

In [15]:
# finding a song I want to use as an input
tay = df[df['artists'] == 'TAYLOR SWIFT'].sort_values('popularity', ascending=False)

In [10]:
# target set will be both artist and name
y_set = ['artists', 'name', 'url']

# droping target from data matrix
df_data = df.drop(y_set, axis=1)

# set target
df_target = df[y_set]

In [11]:
# fit on data, 12 neighbors
nn = NearestNeighbors(algorithm='brute', leaf_size =15, n_neighbors=12, n_jobs=-1)
nn.fit(df_data)

NearestNeighbors(algorithm='brute', leaf_size=15, metric='minkowski',
                 metric_params=None, n_jobs=-1, n_neighbors=12, p=2,
                 radius=1.0)

In [22]:
# sample a song(index) from df_data to use as our query point 
input_index = 141035 # Drake - Tootsie slide

# vectorize 
data_vect = [df_data.iloc[input_index].values]
data_vect

[array([ 2.01000e+03,  1.86000e-01,  5.52000e-01,  3.02253e+05,
         6.08000e-01,  0.00000e+00,  0.00000e+00,  1.00000e+00,
         1.25000e-01, -5.29000e+00,  0.00000e+00,  4.80000e+01,
         2.59000e-02,  1.34050e+02,  1.70000e-01])]

In [23]:
# Query Using kneighbors 
neigh_dist, neigh_indices = nn.kneighbors(data_vect)

In [24]:
# top 12 closest data vectors to our reference vector, data_vect
neigh_dist

array([[ 0.        ,  0.        , 23.87497923, 25.47073031, 31.47119931,
        36.87378797, 37.85822914, 38.90274363, 40.60870203, 41.00981823,
        41.97287928, 43.9201857 ]])

In [22]:
# these are the corresponding indicies of the most similar vectors 
neigh_indices

array([[ 58286,  93860,  55254, 107239,  15938,  56532, 155227,  33730,
         92082, 105990, 120108,  70300]], dtype=int64)

In [23]:
# breaking down the array to a list
indexs = neigh_indices.flat[0:12].tolist()
indexs

[58286,
 93860,
 55254,
 107239,
 15938,
 56532,
 155227,
 33730,
 92082,
 105990,
 120108,
 70300]

In [24]:
# result metrics
df_data.iloc[indexs]

,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
58286,2020,0.3210,0.834,247059,0.454,1,0.000006,1,0.1140,-9.750,0,83,0.2010,81.618,0.8370
93860,2020,0.2890,0.830,247059,0.490,1,0.000003,1,0.1130,-8.820,0,78,0.2090,81.604,0.8450
55254,2005,0.6190,0.520,247053,0.379,0,0.000002,0,0.1080,-9.906,1,57,0.0279,85.934,0.2080
107239,2004,0.1180,0.680,247040,0.659,0,0.000000,0,0.1070,-6.855,1,53,0.2110,91.259,0.4110
15938,2000,0.5900,0.266,247040,0.333,0,0.000000,10,0.1210,-9.876,1,62,0.0310,109.479,0.0796
56532,2011,0.0301,0.226,247027,0.261,0,0.067700,2,0.0628,-11.276,1,57,0.0339,108.915,0.0840
155227,2002,0.5180,0.553,247027,0.530,0,0.000000,4,0.2320,-5.332,0,49,0.0246,84.017,0.1620
33730,1992,0.0380,0.691,247040,0.497,0,0.000006,11,0.0859,-13.248,1,51,0.0473,103.893,0.7010
92082,2011,0.8460,0.414,247053,0.295,0,0.000000,10,0.0790,-9.271,0,53,0.0330,125.833,0.2180
105990,1998,0.1900,0.694,247040,0.368,0,0.000000,8,0.1400,-7.264,1,41,0.0655,60.117,0.4650


In [25]:
# result target
df_target.iloc[indexs]

,artists,name,url
58286,DRAKE,Toosie Slide,http://open.spotify.com/track/127QTOFJsJQp5LbJ...
93860,DRAKE,Toosie Slide,http://open.spotify.com/track/466cKvZn1j45IpxD...
55254,"BRAD PAISLEY', 'DOLLY PARTON",When I Get Where I'm Going (feat. Dolly Parton),http://open.spotify.com/track/3VLCtStwYsAL4LKZ...
107239,"CASSIDY', 'R. KELLY",Hotel (feat. R. Kelly),http://open.spotify.com/track/4hHXhCRSnOKd6nMG...
15938,FAITH HILL,"Where Are You Christmas - From ""Dr. Seuss' How...",http://open.spotify.com/track/1msuiw6pnXYfxZ7E...
56532,M83,Outro,http://open.spotify.com/track/2QVmiA93GVhWNTWQ...
155227,KELLY CHEN,記事本,http://open.spotify.com/track/3FbzN8826gMAaMxU...
33730,THE BELLAMY BROTHERS,Old Hippie,http://open.spotify.com/track/5fv9qtXwNV6Xx3P9...
92082,STAIND,Something to Remind You,http://open.spotify.com/track/4wOQ8upbW1GzHCWj...
105990,DESTINY'S CHILD,"No, No, No, Pt. 1",http://open.spotify.com/track/2pdzseh7ELZCKlXX...


In [27]:
# creates the model into a pickle file
dump(nn, 'model.joblib', compress=True)

['model.joblib']